In [1]:
import os
import pandas as pd
import numpy as np
from haversine import haversine as hs

In [2]:
data = {}
for file in os.listdir('./data_'):
    if '.csv' in file:
        fn = file.replace('.csv','')
        data[fn] = pd.read_csv('./data_/'+file)

In [3]:
for key in data.keys():
    print('======= ' + key + ' ======')
    print(data[key].info())
    print('\n')

======= busan_eschool ======
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Data columns (total 9 columns):
연번           305 non-null int64
구분           305 non-null object
관할지원청        305 non-null object
기관명          305 non-null object
주소           305 non-null object
Latitude     305 non-null float64
Longitude    305 non-null float64
전화           305 non-null object
홈페이지         305 non-null object
dtypes: float64(2), int64(1), object(6)
memory usage: 21.6+ KB
None


======= apt_trade ======
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248503 entries, 0 to 248502
Data columns (total 25 columns):
pk_apt_trade      248503 non-null int64
pk_apt_master     248503 non-null int64
sigungu_cd        248503 non-null int64
dong_cd           248503 non-null int64
deal_ymd          248503 non-null int64
gen_dt            248503 non-null int64
dong              248503 non-null object
bldg_nm           248503 non-null object
bun               248503 non-null object


- '학교이름','학교경도','학교위도','역이름','역경도','역위도' 데이터 프레임 '환경'생성

In [4]:
data['busan_eschool']['lat_lon'] = list(zip(data['busan_eschool']['Latitude'],data['busan_eschool']['Longitude']))
data['busan_eschool'] = data['busan_eschool'][['기관명', 'lat_lon']]
data['busan_eschool'] = data['busan_eschool'].rename(columns = {'기관명':'학교명'})

In [5]:
data['busan_eschool'].head()

,학교명,lat_lon
0,감정초등학교,"(35.096870200000005, 129.0110047)"
1,감천초등학교,"(35.08309620000001, 129.0076053)"
2,광일초등학교,"(35.10372520000001, 129.02986090000002)"
3,괴정초등학교,"(35.1037635, 128.9998745)"
4,구덕초등학교,"(35.1202619, 129.0123589)"


In [6]:
data['busan_subway']['lat_lon'] = list(zip(data['busan_subway']['Latitude'],data['busan_subway']['Longitude']))
data['busan_subway'] = data['busan_subway'][['역명', 'lat_lon']]

In [7]:
data['busan_subway'].head()

,역명,lat_lon
0,다대포해수욕장역,"(35.074697799999996, 128.97118840000002)"
1,다대포항역,"(35.074697799999996, 128.97118840000002)"
2,낫개역,"(35.074697799999996, 128.97118840000002)"
3,신장림역,"(35.074697799999996, 128.97118840000002)"
4,장림역,"(35.074697799999996, 128.97118840000002)"


- 'apt_master' 데이터 프레임에서 필요한 열만 추출

In [8]:
data['apt_master']['lat_lon'] = list(zip(data['apt_master']['lat'],data['apt_master']['lon']))
data['apt_master'] = data['apt_master'][['pk_apt_master','total_house_cnt','total_dong_cnt','max_jisang_floor','min_jisang_floor','total_jucha_rate','heat_method','lat_lon','bldg_cd','gen_dt']]
data['apt_master'].head()

,pk_apt_master,total_house_cnt,total_dong_cnt,max_jisang_floor,min_jisang_floor,total_jucha_rate,heat_method,lat_lon,bldg_cd,gen_dt
0,24,450,3,20,1.0,1.04,개별난방,"(35.160214286423155, 129.02247008783408)",1,1992.12
1,45,788,13,25,18.0,1.06,지역난방,"(35.17814223752016, 129.18080133654178)",1,1998.11
2,49,938,12,25,8.0,1.00,지역난방,"(35.17511010074104, 129.17683514653672)",1,1997.09
3,59,655,11,24,16.0,1.00,지역난방,"(35.177678699197266, 129.16727161464672)",1,1997.05
4,69,1358,19,24,19.0,1.13,지역난방,"(35.16660212799623, 129.18570786561207)",1,1996.11


In [9]:
def min_eschool(lat_lon):
    m = np.inf
    nm = ''
    n = len(data['busan_eschool'])
    for i in range(n):
        d = hs(lat_lon, data['busan_eschool'].iloc[i,1])
        if d < m:
            m = d
            nm = data['busan_eschool'].iloc[i,0]
    return nm, d 

In [10]:
%time data['apt_master']['min_eschool'] = data['apt_master']['lat_lon'].apply(min_eschool)

CPU times: user 14 s, sys: 55.1 ms, total: 14.1 s
Wall time: 14.3 s


In [11]:
def min_subway(lat_lon):
    m = np.inf
    nm = ''
    n = len(data['busan_subway'])
    for i in range(n):
        d = hs(lat_lon, data['busan_subway'].iloc[i,1])
        if d < m:
            m = d
            nm = data['busan_subway'].iloc[i,0]
    return nm, d 

In [12]:
%time data['apt_master']['min_subway'] = data['apt_master']['lat_lon'].apply(min_subway)

CPU times: user 5.54 s, sys: 15.2 ms, total: 5.56 s
Wall time: 5.63 s


In [13]:
data['apt_master'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2770 entries, 0 to 2769
Data columns (total 12 columns):
pk_apt_master       2770 non-null int64
total_house_cnt     2770 non-null int64
total_dong_cnt      2770 non-null int64
max_jisang_floor    2770 non-null int64
min_jisang_floor    2624 non-null float64
total_jucha_rate    2769 non-null float64
heat_method         2614 non-null object
lat_lon             2770 non-null object
bldg_cd             2770 non-null int64
gen_dt              2770 non-null float64
min_eschool         2770 non-null object
min_subway          2770 non-null object
dtypes: float64(3), int64(5), object(4)
memory usage: 259.8+ KB


In [14]:
data['apt_master'] = pd.merge(data['apt_master'], 
              pd.DataFrame(data['apt_master']['min_eschool'].tolist(), 
                           columns=['nm_sch','dist_sch'],
                           index = data['apt_master'].index),
              left_index=True, 
              right_index=True,
              how='left')
data['apt_master'] = pd.merge(data['apt_master'], 
              pd.DataFrame(data['apt_master']['min_subway'].tolist(), 
                           columns=['nm_sub','dist_sub'],
                           index = data['apt_master'].index),
              left_index=True, 
              right_index=True,
              how='left')

In [15]:
data['apt_master'] = data['apt_master'][['pk_apt_master', 'total_house_cnt', 'total_dong_cnt', 'max_jisang_floor', 
                                         'min_jisang_floor', 'total_jucha_rate', 'heat_method','bldg_cd', 'gen_dt',
                                         'nm_sch', 'dist_sch', 'nm_sub', 'dist_sub']]

In [16]:
data['apt_master'].head()

,pk_apt_master,total_house_cnt,total_dong_cnt,max_jisang_floor,min_jisang_floor,total_jucha_rate,heat_method,bldg_cd,gen_dt,nm_sch,dist_sch,nm_sub,dist_sub
0,24,450,3,20,1.0,1.04,개별난방,1,1992.12,개화초등학교,7.662802,동의대,16.087784
1,45,788,13,25,18.0,1.06,지역난방,1,1998.11,부흥초등학교,7.092371,장산,6.653143
2,49,938,12,25,8.0,1.00,지역난방,1,1997.09,부흥초등학교,6.647549,장산,6.956514
3,59,655,11,24,16.0,1.00,지역난방,1,1997.05,양운초등학교,5.918199,장산,6.672089
4,69,1358,19,24,19.0,1.13,지역난방,1,1996.11,좌산초등학교,7.241903,중동,7.985490


- 'apt_trade'를 원하는 피쳐만 모으자

In [17]:
data['apt_trade'] = data['apt_trade'][['pk_apt_trade','pk_apt_master','sigungu_cd','dong_cd','deal_ymd','private_extent','price','floor','pk_apt_detail']]

- 'apt_trade'에 'pk_apt_master'키를 기준으로 'apt_master'정보를 합치자

In [18]:
data['apt_trade'] = pd.merge(data['apt_trade'],data['apt_master'],
                             how='left',on='pk_apt_master')

In [19]:
data['apt_trade'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 248503 entries, 0 to 248502
Data columns (total 21 columns):
pk_apt_trade        248503 non-null int64
pk_apt_master       248503 non-null int64
sigungu_cd          248503 non-null int64
dong_cd             248503 non-null int64
deal_ymd            248503 non-null int64
private_extent      248503 non-null float64
price               248503 non-null int64
floor               248503 non-null int64
pk_apt_detail       248503 non-null int64
total_house_cnt     248503 non-null int64
total_dong_cnt      248503 non-null int64
max_jisang_floor    248503 non-null int64
min_jisang_floor    247717 non-null float64
total_jucha_rate    248493 non-null float64
heat_method         246297 non-null object
bldg_cd             248503 non-null int64
gen_dt              248503 non-null float64
nm_sch              248503 non-null object
dist_sch            248503 non-null float64
nm_sub              248503 non-null object
dist_sub            248503 non-null 

In [20]:
for col in data['apt_trade'].columns:
    if sum(data['apt_trade'][col].isnull()):
        print(str(col) +'열의 %.1f%% 데이터가 null\n' % ( (sum(data['apt_trade'][col].isnull())/len(data['apt_trade'][col]))*100 ))

min_jisang_floor열의 0.3% 데이터가 null

total_jucha_rate열의 0.0% 데이터가 null

heat_method열의 0.9% 데이터가 null



- null 데이터가 있으면 삭제하자

In [21]:
data['apt_trade'] = data['apt_trade'].dropna()

In [22]:
for col in data['apt_trade'].columns:
    if sum(data['apt_trade'][col].isnull()):
        print(str(col) +'열의 %.1f%% 데이터가 null\n' % ( (sum(data['apt_trade'][col].isnull())/len(data['apt_trade'][col]))*100 ))

In [23]:
data['apt_trade'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 246111 entries, 0 to 248502
Data columns (total 21 columns):
pk_apt_trade        246111 non-null int64
pk_apt_master       246111 non-null int64
sigungu_cd          246111 non-null int64
dong_cd             246111 non-null int64
deal_ymd            246111 non-null int64
private_extent      246111 non-null float64
price               246111 non-null int64
floor               246111 non-null int64
pk_apt_detail       246111 non-null int64
total_house_cnt     246111 non-null int64
total_dong_cnt      246111 non-null int64
max_jisang_floor    246111 non-null int64
min_jisang_floor    246111 non-null float64
total_jucha_rate    246111 non-null float64
heat_method         246111 non-null object
bldg_cd             246111 non-null int64
gen_dt              246111 non-null float64
nm_sch              246111 non-null object
dist_sch            246111 non-null float64
nm_sub              246111 non-null object
dist_sub            246111 non-null 

- 데이터 타입을 정제하자

In [24]:
data['apt_trade'].pk_apt_trade = data['apt_trade'].pk_apt_trade.astype(str)
data['apt_trade'].pk_apt_master = data['apt_trade'].pk_apt_master.astype(str)
data['apt_trade'].sigungu_cd = data['apt_trade'].sigungu_cd.astype(str)
data['apt_trade'].dong_cd = data['apt_trade'].dong_cd.astype(str)
data['apt_trade'].floor = data['apt_trade'].floor.astype(str)
data['apt_trade'].pk_apt_detail = data['apt_trade'].pk_apt_detail.astype(str)
data['apt_trade'].bldg_cd = data['apt_trade'].bldg_cd.astype(str)

In [25]:
data['apt_trade']['price'] = data['apt_trade']['price'].astype(np.float)
data['apt_trade']['min_jisang_floor'] = data['apt_trade']['min_jisang_floor'].astype(np.int)

data['apt_trade'].gen_dt = data['apt_trade'].gen_dt.astype(str).str.replace('.','').astype(np.int)

In [26]:
data['apt_trade'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 246111 entries, 0 to 248502
Data columns (total 21 columns):
pk_apt_trade        246111 non-null object
pk_apt_master       246111 non-null object
sigungu_cd          246111 non-null object
dong_cd             246111 non-null object
deal_ymd            246111 non-null int64
private_extent      246111 non-null float64
price               246111 non-null float64
floor               246111 non-null object
pk_apt_detail       246111 non-null object
total_house_cnt     246111 non-null int64
total_dong_cnt      246111 non-null int64
max_jisang_floor    246111 non-null int64
min_jisang_floor    246111 non-null int64
total_jucha_rate    246111 non-null float64
heat_method         246111 non-null object
bldg_cd             246111 non-null object
gen_dt              246111 non-null int64
nm_sch              246111 non-null object
dist_sch            246111 non-null float64
nm_sub              246111 non-null object
dist_sub            246111 non-

In [39]:
data['apt_trade'].to_csv('apt_trade_pre.csv', index=False)

- 여기서부터 정리된 데이터를 **df**로 부르자
- 각 피쳐의 값 갯수를 확인하자

In [44]:
# df = pd.read_csv('apt_trade_pre.csv')
df = data['apt_trade'].copy()

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 246111 entries, 0 to 248502
Data columns (total 21 columns):
pk_apt_trade        246111 non-null object
pk_apt_master       246111 non-null object
sigungu_cd          246111 non-null object
dong_cd             246111 non-null object
deal_ymd            246111 non-null int64
private_extent      246111 non-null float64
price               246111 non-null float64
floor               246111 non-null object
pk_apt_detail       246111 non-null object
total_house_cnt     246111 non-null int64
total_dong_cnt      246111 non-null int64
max_jisang_floor    246111 non-null int64
min_jisang_floor    246111 non-null int64
total_jucha_rate    246111 non-null float64
heat_method         246111 non-null object
bldg_cd             246111 non-null object
gen_dt              246111 non-null int64
nm_sch              246111 non-null object
dist_sch            246111 non-null float64
nm_sub              246111 non-null object
dist_sub            246111 non-

In [46]:
for col in df.columns:
    print('======== ' + col + ' ========')
    print(len(df[col].unique()))

======== pk_apt_trade ========
246111
======== pk_apt_master ========
2047
======== sigungu_cd ========
16
======== dong_cd ========
43
======== deal_ymd ========
77
======== private_extent ========
4059
======== price ========
6275
======== floor ========
82
======== pk_apt_detail ========
6019
======== total_house_cnt ========
739
======== total_dong_cnt ========
35
======== max_jisang_floor ========
55
======== min_jisang_floor ========
44
======== total_jucha_rate ========
203
======== heat_method ========
3
======== bldg_cd ========
2
======== gen_dt ========
509
======== nm_sch ========
275
======== dist_sch ========
2047
======== nm_sub ========
62
======== dist_sub ========
2047


- get_dummies 함수를 이용해서 특정 피쳐를 원핫 인코딩 하자

In [47]:
df.columns

Index(['pk_apt_trade', 'pk_apt_master', 'sigungu_cd', 'dong_cd', 'deal_ymd',
       'private_extent', 'price', 'floor', 'pk_apt_detail', 'total_house_cnt',
       'total_dong_cnt', 'max_jisang_floor', 'min_jisang_floor',
       'total_jucha_rate', 'heat_method', 'bldg_cd', 'gen_dt', 'nm_sch',
       'dist_sch', 'nm_sub', 'dist_sub'],
      dtype='object')

In [49]:
%time df_onehot = pd.get_dummies(df, columns = ['sigungu_cd', 'dong_cd', 'floor', 'heat_method', 'bldg_cd', 'gen_dt', 'nm_sch', 'nm_sub'])

CPU times: user 4.3 s, sys: 332 ms, total: 4.63 s
Wall time: 4.85 s


In [51]:
%time df_onehot.to_csv('apt_trade_onehot.csv', index=False)

CPU times: user 5min 12s, sys: 16.5 s, total: 5min 28s
Wall time: 6min 29s


In [53]:
df_onehot.shape

(246111, 1005)